In [14]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install bitsandbytes
!pip install accelerate

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

def get_k_most_similar_texts_by_tfidf(target_text, texts=None, k=5):
    texts = []
    file_path = "/content/gdrive/MyDrive/News-Data/train.csv"
    df = pd.read_csv(file_path, on_bad_lines='skip')
    for index, row in df.iterrows():
        texts.append((row[1], row[2], row[4]))

    # Initialize the vectorizer to include unigrams, bigrams, and trigrams
    vectorizer = TfidfVectorizer(ngram_range=(1, 3))
    text_vectors = vectorizer.fit_transform([text[0] for text in texts] + [target_text])

    # Calculate cosine similarity
    cosine_similarities = cosine_similarity(text_vectors[-1], text_vectors[:-1])
    cosine_similarities = cosine_similarities[0]  # Extract the first row from the 2D array

    # Get indices of top k similar texts
    top_indices = cosine_similarities.argsort()[::-1][:k]

    # Return the top k similar texts and their similarities
    results = [(texts[i][0], texts[i][-1], cosine_similarities[i]) for i in top_indices]
    return results

# Class of large language model used
class AYA23Generator:
    def __init__(self, model_name, quantize_4bit=True, use_flash_attention=False):
        self.model_name = model_name
        self.quantize_4bit = quantize_4bit
        self.use_flash_attention = use_flash_attention
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = None
        self.tokenizer = None
        self._load_model()

    def _load_model(self):
        # quantization config that can be used if it is needed
        quantization_config = None
        if self.quantize_4bit:
            quantization_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_use_double_quant=True,
                bnb_4bit_compute_dtype=torch.bfloat16,

            )


        attn_implementation = None
        if self.use_flash_attention:
            attn_implementation = "flash_attention_2"

        # Loading the model
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            quantization_config=quantization_config,
            attn_implementation=attn_implementation,
            torch_dtype=torch.bfloat16,
            device_map="auto",
        )
        # Loading the tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        print("Model and tokenizer loaded successfully.")

    # Formatting the prompt in the desired format
    def get_message_format(self, prompts):
        return [[{"role": "user", "content": p}] for p in prompts]

    # Generating the output with the generation config provided
    def generate_responses(self, prompts, temperature=0.3, top_p=0.75, top_k=0, max_new_tokens=1024):
        messages = self.get_message_format(prompts)
        input_ids = self.tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            padding=True,
            return_tensors="pt",
        ).to(self.device)
        prompt_padded_len = len(input_ids[0])
        gen_tokens = self.model.generate(
            input_ids,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            max_new_tokens=max_new_tokens,
            do_sample=True,
        )
        gen_tokens = [gt[prompt_padded_len:] for gt in gen_tokens]
        return self.tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)

import sys
sys.path.append('/content')
MODEL_NAME = "CohereForAI/aya-23-8B"
generator = AYA23Generator(MODEL_NAME)
test_df = pd.read_csv('/content/gdrive/MyDrive/News-Data/test.csv')
aya_df = pd.read_csv('/content/gdrive/MyDrive/News-Data/results_aya.csv')
column_name_to_write = 'predicted_k_50'
start_row = aya_df.index[pd.isna(aya_df[column_name_to_write])].tolist()[0]
for i in range(start_row, len(aya_df)):
      # Specifying the k_shot value, and the correct prompt text accordingly
      k_shot = int(column_name_to_write.split('_')[2])
      if k_shot == 0:
          prompt_fa_kshot = """هدف، داشتن یک دسته‌بند دودویی است که با گرفتن هر متن ورودی، کلاس آن را در خروجی مشخص می‌کند. کلاس‌ها شامل دو دسته‌ی 1 یا 0 هستند. 1 یعنی خبر مهم است و 0 یعنی خبر مهم نیست.

      شرح تسک:
      متن یا خبری را مهم یا تاثیرگذار می‌گوییم اگر که برای بیش‌تر کاربران فارسی‌زبان اهمیت بالایی داشته باشد. یا به عبارت دیگر، جمعیت زیاد و بزرگی از ایرانیان مایل باشند که آن متن یا خبر را بخوانند و یا برای یکدیگر بفرستند. اگر خبری مربوط به یک قشر کوچک یا جامعه‌ی خاصی از کاربران باشد، آن خبر مهم نیست.
      در صورتی که متن ورودی مهم باشد، کلاس 1 خواهد بود و در صورتی که مهم نباشد، کلاس 0 خواهد بود
      برخی از مفاهیم مهم عبارت‌اند از:
      یارانه و سهام و مواردی که قرار است پول به مردم برسد مهم هستند
      ثبت نام مسکن و خانه و اخبار مربوط به وام‌ها و...
      ثبت نام خودرو
      افزایش و کاهش های شدید و زیاد قیمت ارز یا طلا و سکه و یا تورم

      سیاسی:
      اخبار جنگ، برجام، توافق های ایران،
      تحریم های ایران،
      خبرهای جنگ‌های بزرگ منطقه‌ای،
      عزل و نصب مقامات بلندپایه ایرانی،
      این‌ها همگی مهم هستند

      ورزشی:
      اخبار مربوط به تیم‌های معروف و پرطرفدار ایرانی و همین‌طور اروپایی مهم است

        با توجه به متن زیر تنها در یک واژه پاسخ بده که باتوجه به مفاهیمی که در بالا مطرح شد و قدرت استنتاجی که خودت داری، آیا متن
      مهم (تاثیرگذاری) حساب می‌شود یا خیر. (1 یا 0):
      '''
      ^^body^^
      '''
      در خروجی فقط مجاز هستی عدد ۱ یا عدد ۰ بنویسی. بدون هیچ توضیح اضافه‌ای.
      """
      else:
          prompt_fa_kshot = """هدف، داشتن یک دسته‌بند دودویی است که با گرفتن هر متن ورودی، کلاس آن را در خروجی مشخص می‌کند. کلاس‌ها شامل دو دسته‌ی 1 یا 0 هستند. 1 یعنی خبر مهم است و 0 یعنی خبر مهم نیست.

          شرح تسک:
          متن یا خبری را مهم یا تاثیرگذار می‌گوییم اگر که برای بیش‌تر کاربران فارسی‌زبان اهمیت بالایی داشته باشد. یا به عبارت دیگر، جمعیت زیاد و بزرگی از ایرانیان مایل باشند که آن متن یا خبر را بخوانند و یا برای یکدیگر بفرستند. اگر خبری مربوط به یک قشر کوچک یا جامعه‌ی خاصی از کاربران باشد، آن خبر مهم نیست.
          در صورتی که متن ورودی مهم باشد، کلاس 1 خواهد بود و در صورتی که مهم نباشد، کلاس 0 خواهد بود
          برخی از مفاهیم مهم عبارت‌اند از:
          یارانه و سهام و مواردی که قرار است پول به مردم برسد مهم هستند
          ثبت نام مسکن و خانه و اخبار مربوط به وام‌ها و...
          ثبت نام خودرو
          افزایش و کاهش های شدید و زیاد قیمت ارز یا طلا و سکه و یا تورم

          سیاسی:
          اخبار جنگ، برجام، توافق های ایران،
          تحریم های ایران،
          خبرهای جنگ‌های بزرگ منطقه‌ای،
          عزل و نصب مقامات بلندپایه ایرانی،
          این‌ها همگی مهم هستند

          ورزشی:
          اخبار مربوط به تیم‌های معروف و پرطرفدار ایرانی و همین‌طور اروپایی مهم است

          نمونه‌ها: چند نمونه پایین را ببین و باتوجه به آن‌ها به سوال پایین پاسخ بده
          SAMPLES_HERE
          از روی نمونه‌های بالایی یاد بگیر و خروجی را مشخص کن (فقط ۰ یا ۱).
          حال  با توجه به «نمونه‌های بالا»، برای متن زیر تنها در یک واژه پاسخ بده که باتوجه به مفاهیمی که در بالا مطرح شد و قدرت استنتاجی که خودت داری، آیا متن
          مهم (تاثیرگذاری) حساب می‌شود یا خیر. (1 یا 0):
          '''
          ^^body^^
          '''
          در خروجی فقط مجاز هستی عدد ۱ یا عدد ۰ بنویسی. بدون هیچ توضیح اضافه‌ای.
          """
      test_df_counter = i % len(test_df)
      print(f"test_df_counter is {test_df_counter}")
      target_text = aya_df['text'][i]
      if (len(target_text) > 10000):
          target_text = target_text[:8000]
    
      # Placing the target text and similar news in their correct position in the prompt template
      new_prompt = prompt_fa_kshot.replace("^^body^^",  target_text)
      if k_shot != 0:
          sample_str = ''
          for _ in range(k_shot):
              sample_str += 'متن: {}\n' +\
              'خروجی : {}\n'
          new_prompt = new_prompt.replace('SAMPLES_HERE', sample_str)
          samples = []
          similar_texts = get_k_most_similar_texts_by_tfidf(test_df['title'][test_df_counter] + '\n' + test_df['text'][test_df_counter], k=k_shot)
          for text in similar_texts:
              samples.append(text[0])
              samples.append(text[1])
          new_prompt = new_prompt.format(*samples)
      new_prompt = [new_prompt]
      # Providing the prompt to the model and generating the output
      result = int(generator.generate_responses(new_prompt)[0])
      torch.cuda.empty_cache()
      # Saving the output to the working dataframe
      aya_df.at[i, column_name_to_write] = result
      print(f"answer of row {i} is {result} and k is {k_shot} and text type is {aya_df['text_type'][i]}")
      if i % 10 == 0:
          aya_df.to_csv('/content/gdrive/MyDrive/News-Data/results_aya.csv', index=False)
          print(f"dataframe saved to csv file at iteration {i}")

aya_df.to_csv('/content/gdrive/MyDrive/News-Data/results_aya.csv', index=False)